# Data Science Study Group Project: Neutrino type and background classification
# Building your first fully connected network and a CNN 

As last time launch a job - but now use `jupyter_job_GPU.sh` - this will request a GPU

Also before the meeting please copy the dataset to the local SLURM temporary fast storage

Ok let's make a dataset:

In [2]:
!hostname
!cp /scratch/wfedorko/TRIUMF_DS_NUPRISM/merged_IWCDmPMT_varyE.h5 ${SLURM_TMPDIR}/ -v

cdr906.int.cedar.computecanada.ca
'/scratch/wfedorko/TRIUMF_DS_NUPRISM/merged_IWCDmPMT_varyE.h5' -> '/localscratch/wfedorko.22868755.0/merged_IWCDmPMT_varyE.h5'


In [3]:
import os
from io_utils.data_handling import WCH5Dataset
dset=WCH5Dataset(os.environ['SLURM_TMPDIR']+"/merged_IWCDmPMT_varyE.h5",val_split=0.1,test_split=0.1)

Let's first create a configuration object -we'll use this to set up our training engine

In [4]:
#path = /fast_scratch/WatChMaL/data/IWCDmPMT/varyE/IWCDmPMT_varyE_100-1000MeV_300k_fmt.h5

In [5]:
class CONFIG:
    pass
config=CONFIG()
config.batch_size_test = 1024
config.batch_size_train = 32
config.batch_size_val = 1024
config.lr=0.001
config.device = 'gpu'
config.gpu_list = [0]
config.dump_path = 'model_state_dumps'


Now Let's make our model. We'll talk about 
  - model parameters
  - inputs and the forward method
  - Modules containing modules
  - Sequential Module<br>
  Lets open [simpleMLP](../edit/models/simpleMLP.py)

In [6]:
from models.simpleMLP import SimpleMLP

In [7]:
model_MLP=SimpleMLP(num_classes=3)

Let's look at the parameters:

In [8]:
print(model_MLP.parameters)

<bound method Module.parameters of SimpleMLP(
  (relu): ReLU()
  (softmax): Softmax()
  (fc1): Linear(in_features=24320, out_features=9728, bias=True)
  (fc2): Linear(in_features=9728, out_features=4864, bias=True)
  (fc3): Linear(in_features=4864, out_features=1600, bias=True)
  (fc4): Linear(in_features=1600, out_features=400, bias=True)
  (fc5): Linear(in_features=400, out_features=3, bias=True)
)>


Now let's look at sequential version

In [9]:
from models.simpleMLP import SimpleMLPSEQ
model_MLPSEQ=SimpleMLPSEQ(num_classes=3)

Again we can take a look at the parameters - now `Sequential` is a parameter with parameters of it's own

In [10]:
print(model_MLPSEQ.parameters)

<bound method Module.parameters of SimpleMLPSEQ(
  (_sequence): Sequential(
    (0): Linear(in_features=24320, out_features=9728, bias=True)
    (1): ReLU()
    (2): Linear(in_features=9728, out_features=4864, bias=True)
    (3): ReLU()
    (4): Linear(in_features=4864, out_features=1600, bias=True)
    (5): ReLU()
    (6): Linear(in_features=1600, out_features=400, bias=True)
    (7): ReLU()
    (8): Linear(in_features=400, out_features=3, bias=True)
  )
)>


In [11]:
import numpy as np
transform=np.ravel
dset=WCH5Dataset(os.environ['SLURM_TMPDIR']+"/merged_IWCDmPMT_varyE.h5",val_split=0.1,test_split=0.1,transform=transform)

Let's take a look at the 
[training engine](../edit/training_utils/engine.py) <br><br>
We will talk about:
   - fetching data from the dataloader and sending it to GPU
   - designing the forward method and passing the input data through the model
   - the backward pass, gradient evaluation and a step in the parameter space
   - validation loop

In [14]:
from training_utils.engine import Engine
!mkdir model_state_dumps

In [15]:
engine=Engine(model_MLP,dset,config)

Requesting GPUs. GPU list : [0]
Main GPU: cuda:0
CUDA is available
Creating a directory for run dump: model_state_dumps/20190619_211845/


In [16]:
print(vars(config))

{'batch_size_test': 1024, 'batch_size_train': 32, 'batch_size_val': 1024, 'lr': 0.001, 'device': 'gpu', 'gpu_list': [0], 'dump_path': 'model_state_dumps'}


Ok let's trainthe MLP!!!

In [ ]:
engine.train(epochs=5,report_interval=10,valid_interval=100)

Epoch 0 Starting @ 2019-06-19 21:19:08
... Iteration 1 ... Epoch 0.00 ... Loss 3.244 ... Accuracy 0.219
... Iteration 10 ... Epoch 0.00 ... Loss 1.628 ... Accuracy 0.344
... Iteration 20 ... Epoch 0.00 ... Loss 0.980 ... Accuracy 0.562
... Iteration 30 ... Epoch 0.00 ... Loss 1.098 ... Accuracy 0.469
... Iteration 40 ... Epoch 0.00 ... Loss 1.080 ... Accuracy 0.344
... Iteration 50 ... Epoch 0.00 ... Loss 1.092 ... Accuracy 0.375
... Iteration 60 ... Epoch 0.00 ... Loss 0.973 ... Accuracy 0.500
... Iteration 70 ... Epoch 0.00 ... Loss 1.252 ... Accuracy 0.406
... Iteration 80 ... Epoch 0.00 ... Loss 0.810 ... Accuracy 0.688
... Iteration 90 ... Epoch 0.00 ... Loss 1.100 ... Accuracy 0.406
... Iteration 100 ... Epoch 0.00 ... Loss 0.776 ... Accuracy 0.781
Saved checkpoint as: model_state_dumps/20190619_211845/SimpleMLP.pth
best validation loss so far!: 0.8180537819862366
Saved checkpoint as: model_state_dumps/20190619_211845/SimpleMLPBEST.pth
... Iteration 110 ... Epoch 0.00 ... Loss 0.

Saved checkpoint as: model_state_dumps/20190619_211845/SimpleMLPBEST.pth
... Iteration 1010 ... Epoch 0.04 ... Loss 0.572 ... Accuracy 0.750
... Iteration 1020 ... Epoch 0.05 ... Loss 0.615 ... Accuracy 0.562
... Iteration 1030 ... Epoch 0.05 ... Loss 0.315 ... Accuracy 0.844
... Iteration 1040 ... Epoch 0.05 ... Loss 0.438 ... Accuracy 0.750
... Iteration 1050 ... Epoch 0.05 ... Loss 0.472 ... Accuracy 0.625
... Iteration 1060 ... Epoch 0.05 ... Loss 0.428 ... Accuracy 0.750
... Iteration 1070 ... Epoch 0.05 ... Loss 0.394 ... Accuracy 0.781
... Iteration 1080 ... Epoch 0.05 ... Loss 0.442 ... Accuracy 0.781
... Iteration 1090 ... Epoch 0.05 ... Loss 0.560 ... Accuracy 0.625
... Iteration 1100 ... Epoch 0.05 ... Loss 0.437 ... Accuracy 0.750
Saved checkpoint as: model_state_dumps/20190619_211845/SimpleMLP.pth
... Iteration 1110 ... Epoch 0.05 ... Loss 0.392 ... Accuracy 0.812
... Iteration 1120 ... Epoch 0.05 ... Loss 0.445 ... Accuracy 0.688
... Iteration 1130 ... Epoch 0.05 ... Loss

... Iteration 2070 ... Epoch 0.09 ... Loss 0.397 ... Accuracy 0.750
... Iteration 2080 ... Epoch 0.09 ... Loss 0.434 ... Accuracy 0.781
... Iteration 2090 ... Epoch 0.09 ... Loss 0.650 ... Accuracy 0.656
... Iteration 2100 ... Epoch 0.09 ... Loss 0.357 ... Accuracy 0.781
Saved checkpoint as: model_state_dumps/20190619_211845/SimpleMLP.pth
... Iteration 2110 ... Epoch 0.09 ... Loss 0.452 ... Accuracy 0.719
... Iteration 2120 ... Epoch 0.09 ... Loss 0.472 ... Accuracy 0.656
... Iteration 2130 ... Epoch 0.09 ... Loss 0.516 ... Accuracy 0.656
... Iteration 2140 ... Epoch 0.10 ... Loss 0.379 ... Accuracy 0.812
... Iteration 2150 ... Epoch 0.10 ... Loss 0.442 ... Accuracy 0.781
... Iteration 2160 ... Epoch 0.10 ... Loss 0.509 ... Accuracy 0.719
... Iteration 2170 ... Epoch 0.10 ... Loss 0.436 ... Accuracy 0.812
... Iteration 2180 ... Epoch 0.10 ... Loss 0.479 ... Accuracy 0.688
... Iteration 2190 ... Epoch 0.10 ... Loss 0.416 ... Accuracy 0.812
... Iteration 2200 ... Epoch 0.10 ... Loss 0.53

... Iteration 3130 ... Epoch 0.14 ... Loss 0.376 ... Accuracy 0.750
... Iteration 3140 ... Epoch 0.14 ... Loss 0.471 ... Accuracy 0.688
... Iteration 3150 ... Epoch 0.14 ... Loss 0.448 ... Accuracy 0.688
... Iteration 3160 ... Epoch 0.14 ... Loss 0.407 ... Accuracy 0.750
... Iteration 3170 ... Epoch 0.14 ... Loss 0.495 ... Accuracy 0.688
... Iteration 3180 ... Epoch 0.14 ... Loss 0.544 ... Accuracy 0.625
... Iteration 3190 ... Epoch 0.14 ... Loss 0.416 ... Accuracy 0.750
... Iteration 3200 ... Epoch 0.14 ... Loss 0.481 ... Accuracy 0.719
Saved checkpoint as: model_state_dumps/20190619_211845/SimpleMLP.pth
... Iteration 3210 ... Epoch 0.14 ... Loss 0.521 ... Accuracy 0.625
... Iteration 3220 ... Epoch 0.14 ... Loss 0.538 ... Accuracy 0.719
... Iteration 3230 ... Epoch 0.14 ... Loss 0.437 ... Accuracy 0.812
... Iteration 3240 ... Epoch 0.14 ... Loss 0.455 ... Accuracy 0.750
... Iteration 3250 ... Epoch 0.14 ... Loss 0.476 ... Accuracy 0.688
... Iteration 3260 ... Epoch 0.14 ... Loss 0.38

... Iteration 4210 ... Epoch 0.19 ... Loss 0.431 ... Accuracy 0.719
... Iteration 4220 ... Epoch 0.19 ... Loss 0.346 ... Accuracy 0.844
... Iteration 4230 ... Epoch 0.19 ... Loss 0.407 ... Accuracy 0.750
... Iteration 4240 ... Epoch 0.19 ... Loss 0.515 ... Accuracy 0.656
... Iteration 4250 ... Epoch 0.19 ... Loss 0.426 ... Accuracy 0.812
... Iteration 4260 ... Epoch 0.19 ... Loss 0.447 ... Accuracy 0.781
... Iteration 4270 ... Epoch 0.19 ... Loss 0.450 ... Accuracy 0.781
... Iteration 4280 ... Epoch 0.19 ... Loss 0.385 ... Accuracy 0.812
... Iteration 4290 ... Epoch 0.19 ... Loss 0.516 ... Accuracy 0.625
... Iteration 4300 ... Epoch 0.19 ... Loss 0.390 ... Accuracy 0.750
Saved checkpoint as: model_state_dumps/20190619_211845/SimpleMLP.pth
best validation loss so far!: 0.417324423789978
Saved checkpoint as: model_state_dumps/20190619_211845/SimpleMLPBEST.pth
... Iteration 4310 ... Epoch 0.19 ... Loss 0.463 ... Accuracy 0.719
... Iteration 4320 ... Epoch 0.19 ... Loss 0.407 ... Accuracy 

... Iteration 5300 ... Epoch 0.24 ... Loss 0.463 ... Accuracy 0.688
Saved checkpoint as: model_state_dumps/20190619_211845/SimpleMLP.pth


In [13]:
model_MLP._get_name()

'SimpleMLP'

In [14]:
from models.simpleCNN import SimpleCNN
model_CNN=SimpleCNN(num_input_channels=38,num_classes=3)

In [15]:
def rotate_chan(x):
    return np.transpose(x,(2,0,1))

In [16]:
dset=WCH5Dataset("/fast_scratch/WatChMaL/data/IWCDmPMT/varyE/IWCDmPMT_varyE_100-1000MeV_300k_fmt.h5",val_split=0.1,test_split=0.1,transform=rotate_chan)

In [17]:
engine=Engine(model_CNN,dset,config)

Requesting GPUs. GPU list : [0]
Main GPU: cuda:0
CUDA is available
Creating a directory for run dump: model_state_dumps/20190619_194634/


In [18]:
model_CNN.parameters

<bound method Module.parameters of SimpleCNN(
  (relu): ReLU()
  (softmax): Softmax()
  (f_embed): Conv2d(38, 64, kernel_size=(1, 1), stride=(1, 1))
  (f_conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (f_max_pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (f_conv2a): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (f_conv2b): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (f_max_pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (f_conv3a): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (f_conv3b): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (f_max_pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (f_conv4): Conv2d(128, 128, kernel_size=(2, 2), stride=(1, 1))
  (fc1): Linear(in_features=128, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=

In [19]:
engine.train(epochs=5,report_interval=10,valid_interval=100)

Epoch 0 Starting @ 2019-06-19 19:46:39
... Iteration 1 ... Epoch 0.00 ... Loss 1.097 ... Accuracy 0.375
... Iteration 10 ... Epoch 0.00 ... Loss 1.088 ... Accuracy 0.531
... Iteration 20 ... Epoch 0.00 ... Loss 1.076 ... Accuracy 0.469
... Iteration 30 ... Epoch 0.00 ... Loss 1.088 ... Accuracy 0.406
... Iteration 40 ... Epoch 0.00 ... Loss 1.088 ... Accuracy 0.406
... Iteration 50 ... Epoch 0.00 ... Loss 1.071 ... Accuracy 0.406
... Iteration 60 ... Epoch 0.00 ... Loss 1.079 ... Accuracy 0.281
... Iteration 70 ... Epoch 0.00 ... Loss 1.047 ... Accuracy 0.469
... Iteration 80 ... Epoch 0.00 ... Loss 1.088 ... Accuracy 0.375
... Iteration 90 ... Epoch 0.00 ... Loss 1.091 ... Accuracy 0.375
... Iteration 100 ... Epoch 0.00 ... Loss 1.092 ... Accuracy 0.188
Saved checkpoint as: model_state_dumps/20190619_194634/SimpleCNN.pth
best validation loss so far!: 1.0666351318359375
Saved checkpoint as: model_state_dumps/20190619_194634/SimpleCNNBEST.pth
... Iteration 110 ... Epoch 0.00 ... Loss 1.

... Iteration 1000 ... Epoch 0.04 ... Loss 0.752 ... Accuracy 0.688
Saved checkpoint as: model_state_dumps/20190619_194634/SimpleCNN.pth
best validation loss so far!: 0.5528263449668884
Saved checkpoint as: model_state_dumps/20190619_194634/SimpleCNNBEST.pth
... Iteration 1010 ... Epoch 0.04 ... Loss 0.683 ... Accuracy 0.594
... Iteration 1020 ... Epoch 0.05 ... Loss 0.493 ... Accuracy 0.844
... Iteration 1030 ... Epoch 0.05 ... Loss 0.681 ... Accuracy 0.594
... Iteration 1040 ... Epoch 0.05 ... Loss 0.668 ... Accuracy 0.625
... Iteration 1050 ... Epoch 0.05 ... Loss 0.473 ... Accuracy 0.812
... Iteration 1060 ... Epoch 0.05 ... Loss 0.692 ... Accuracy 0.625
... Iteration 1070 ... Epoch 0.05 ... Loss 0.436 ... Accuracy 0.812
... Iteration 1080 ... Epoch 0.05 ... Loss 0.508 ... Accuracy 0.688
... Iteration 1090 ... Epoch 0.05 ... Loss 0.426 ... Accuracy 0.875
... Iteration 1100 ... Epoch 0.05 ... Loss 0.545 ... Accuracy 0.656
Saved checkpoint as: model_state_dumps/20190619_194634/Simple

... Iteration 2030 ... Epoch 0.09 ... Loss 0.323 ... Accuracy 0.938
... Iteration 2040 ... Epoch 0.09 ... Loss 0.546 ... Accuracy 0.594
... Iteration 2050 ... Epoch 0.09 ... Loss 0.447 ... Accuracy 0.656
... Iteration 2060 ... Epoch 0.09 ... Loss 0.507 ... Accuracy 0.688
... Iteration 2070 ... Epoch 0.09 ... Loss 0.528 ... Accuracy 0.688
... Iteration 2080 ... Epoch 0.09 ... Loss 0.480 ... Accuracy 0.750
... Iteration 2090 ... Epoch 0.09 ... Loss 1.013 ... Accuracy 0.375
... Iteration 2100 ... Epoch 0.09 ... Loss 0.661 ... Accuracy 0.781
Saved checkpoint as: model_state_dumps/20190619_194634/SimpleCNN.pth
... Iteration 2110 ... Epoch 0.09 ... Loss 0.650 ... Accuracy 0.781
... Iteration 2120 ... Epoch 0.09 ... Loss 0.633 ... Accuracy 0.656
... Iteration 2130 ... Epoch 0.09 ... Loss 0.553 ... Accuracy 0.656
... Iteration 2140 ... Epoch 0.10 ... Loss 0.560 ... Accuracy 0.594
... Iteration 2150 ... Epoch 0.10 ... Loss 0.527 ... Accuracy 0.750
... Iteration 2160 ... Epoch 0.10 ... Loss 0.55

... Iteration 3110 ... Epoch 0.14 ... Loss 0.474 ... Accuracy 0.781
... Iteration 3120 ... Epoch 0.14 ... Loss 0.468 ... Accuracy 0.719
... Iteration 3130 ... Epoch 0.14 ... Loss 0.385 ... Accuracy 0.812
... Iteration 3140 ... Epoch 0.14 ... Loss 0.467 ... Accuracy 0.781
... Iteration 3150 ... Epoch 0.14 ... Loss 0.481 ... Accuracy 0.719
... Iteration 3160 ... Epoch 0.14 ... Loss 0.460 ... Accuracy 0.781
... Iteration 3170 ... Epoch 0.14 ... Loss 0.230 ... Accuracy 0.969
... Iteration 3180 ... Epoch 0.14 ... Loss 0.548 ... Accuracy 0.781
... Iteration 3190 ... Epoch 0.14 ... Loss 0.412 ... Accuracy 0.750
... Iteration 3200 ... Epoch 0.14 ... Loss 0.362 ... Accuracy 0.812
Saved checkpoint as: model_state_dumps/20190619_194634/SimpleCNN.pth
... Iteration 3210 ... Epoch 0.14 ... Loss 0.331 ... Accuracy 0.812
... Iteration 3220 ... Epoch 0.14 ... Loss 0.514 ... Accuracy 0.750
... Iteration 3230 ... Epoch 0.14 ... Loss 0.564 ... Accuracy 0.625
... Iteration 3240 ... Epoch 0.14 ... Loss 0.42

... Iteration 4180 ... Epoch 0.19 ... Loss 0.496 ... Accuracy 0.625
... Iteration 4190 ... Epoch 0.19 ... Loss 0.491 ... Accuracy 0.719
... Iteration 4200 ... Epoch 0.19 ... Loss 0.567 ... Accuracy 0.562
Saved checkpoint as: model_state_dumps/20190619_194634/SimpleCNN.pth
... Iteration 4210 ... Epoch 0.19 ... Loss 0.352 ... Accuracy 0.844
... Iteration 4220 ... Epoch 0.19 ... Loss 0.443 ... Accuracy 0.781
... Iteration 4230 ... Epoch 0.19 ... Loss 0.403 ... Accuracy 0.812
... Iteration 4240 ... Epoch 0.19 ... Loss 0.498 ... Accuracy 0.688
... Iteration 4250 ... Epoch 0.19 ... Loss 0.379 ... Accuracy 0.844
... Iteration 4260 ... Epoch 0.19 ... Loss 0.451 ... Accuracy 0.719
... Iteration 4270 ... Epoch 0.19 ... Loss 0.471 ... Accuracy 0.750
... Iteration 4280 ... Epoch 0.19 ... Loss 0.446 ... Accuracy 0.781
... Iteration 4290 ... Epoch 0.19 ... Loss 0.496 ... Accuracy 0.688
... Iteration 4300 ... Epoch 0.19 ... Loss 0.283 ... Accuracy 0.875
Saved checkpoint as: model_state_dumps/20190619

...

... Iteration 108040 ... Epoch 4.80 ... Loss 0.250 ... Accuracy 0.938
... Iteration 108050 ... Epoch 4.80 ... Loss 0.649 ... Accuracy 0.688
... Iteration 108060 ... Epoch 4.80 ... Loss 0.361 ... Accuracy 0.875
... Iteration 108070 ... Epoch 4.80 ... Loss 0.481 ... Accuracy 0.719
... Iteration 108080 ... Epoch 4.80 ... Loss 0.381 ... Accuracy 0.844
... Iteration 108090 ... Epoch 4.80 ... Loss 0.385 ... Accuracy 0.719
... Iteration 108100 ... Epoch 4.80 ... Loss 0.378 ... Accuracy 0.875
Saved checkpoint as: model_state_dumps/20190619_194634/SimpleCNN.pth
... Iteration 108110 ... Epoch 4.80 ... Loss 0.351 ... Accuracy 0.812
... Iteration 108120 ... Epoch 4.81 ... Loss 0.379 ... Accuracy 0.781
... Iteration 108130 ... Epoch 4.81 ... Loss 0.526 ... Accuracy 0.719
... Iteration 108140 ... Epoch 4.81 ... Loss 0.345 ... Accuracy 0.844
... Iteration 108150 ... Epoch 4.81 ... Loss 0.313 ... Accuracy 0.844
... Iteration 108160 ... Epoch 4.81 ... Loss 0.561 ... Accuracy 0.812
... Iteration 108170 

... Iteration 109120 ... Epoch 4.85 ... Loss 0.462 ... Accuracy 0.750
... Iteration 109130 ... Epoch 4.85 ... Loss 0.415 ... Accuracy 0.750
... Iteration 109140 ... Epoch 4.85 ... Loss 0.421 ... Accuracy 0.781
... Iteration 109150 ... Epoch 4.85 ... Loss 0.422 ... Accuracy 0.781
... Iteration 109160 ... Epoch 4.85 ... Loss 0.364 ... Accuracy 0.844
... Iteration 109170 ... Epoch 4.85 ... Loss 0.389 ... Accuracy 0.750
... Iteration 109180 ... Epoch 4.85 ... Loss 0.374 ... Accuracy 0.812
... Iteration 109190 ... Epoch 4.85 ... Loss 0.432 ... Accuracy 0.781
... Iteration 109200 ... Epoch 4.85 ... Loss 0.345 ... Accuracy 0.781
Saved checkpoint as: model_state_dumps/20190619_194634/SimpleCNN.pth
... Iteration 109210 ... Epoch 4.85 ... Loss 0.424 ... Accuracy 0.812
... Iteration 109220 ... Epoch 4.85 ... Loss 0.331 ... Accuracy 0.875
... Iteration 109230 ... Epoch 4.85 ... Loss 0.452 ... Accuracy 0.750
... Iteration 109240 ... Epoch 4.86 ... Loss 0.320 ... Accuracy 0.844
... Iteration 109250 

... Iteration 110200 ... Epoch 4.90 ... Loss 0.302 ... Accuracy 0.906
Saved checkpoint as: model_state_dumps/20190619_194634/SimpleCNN.pth
... Iteration 110210 ... Epoch 4.90 ... Loss 0.385 ... Accuracy 0.875
... Iteration 110220 ... Epoch 4.90 ... Loss 0.426 ... Accuracy 0.719
... Iteration 110230 ... Epoch 4.90 ... Loss 0.315 ... Accuracy 0.844
... Iteration 110240 ... Epoch 4.90 ... Loss 0.392 ... Accuracy 0.844
... Iteration 110250 ... Epoch 4.90 ... Loss 0.488 ... Accuracy 0.750
... Iteration 110260 ... Epoch 4.90 ... Loss 0.487 ... Accuracy 0.656
... Iteration 110270 ... Epoch 4.90 ... Loss 0.411 ... Accuracy 0.781
... Iteration 110280 ... Epoch 4.90 ... Loss 0.443 ... Accuracy 0.750
... Iteration 110290 ... Epoch 4.90 ... Loss 0.327 ... Accuracy 0.844
... Iteration 110300 ... Epoch 4.90 ... Loss 0.464 ... Accuracy 0.750
Saved checkpoint as: model_state_dumps/20190619_194634/SimpleCNN.pth
... Iteration 110310 ... Epoch 4.90 ... Loss 0.344 ... Accuracy 0.844
... Iteration 110320 .

... Iteration 111280 ... Epoch 4.95 ... Loss 0.421 ... Accuracy 0.781
... Iteration 111290 ... Epoch 4.95 ... Loss 0.217 ... Accuracy 0.969
... Iteration 111300 ... Epoch 4.95 ... Loss 0.286 ... Accuracy 0.906
Saved checkpoint as: model_state_dumps/20190619_194634/SimpleCNN.pth
... Iteration 111310 ... Epoch 4.95 ... Loss 0.421 ... Accuracy 0.812
... Iteration 111320 ... Epoch 4.95 ... Loss 0.354 ... Accuracy 0.812
... Iteration 111330 ... Epoch 4.95 ... Loss 0.389 ... Accuracy 0.875
... Iteration 111340 ... Epoch 4.95 ... Loss 0.419 ... Accuracy 0.750
... Iteration 111350 ... Epoch 4.95 ... Loss 0.473 ... Accuracy 0.781
... Iteration 111360 ... Epoch 4.95 ... Loss 0.437 ... Accuracy 0.750
... Iteration 111370 ... Epoch 4.95 ... Loss 0.462 ... Accuracy 0.812
... Iteration 111380 ... Epoch 4.95 ... Loss 0.355 ... Accuracy 0.844
... Iteration 111390 ... Epoch 4.95 ... Loss 0.460 ... Accuracy 0.750
... Iteration 111400 ... Epoch 4.95 ... Loss 0.510 ... Accuracy 0.781
Saved checkpoint as: 

... Iteration 112360 ... Epoch 4.99 ... Loss 0.452 ... Accuracy 0.688
... Iteration 112370 ... Epoch 4.99 ... Loss 0.356 ... Accuracy 0.844
... Iteration 112380 ... Epoch 4.99 ... Loss 0.438 ... Accuracy 0.688
... Iteration 112390 ... Epoch 5.00 ... Loss 0.405 ... Accuracy 0.812
... Iteration 112400 ... Epoch 5.00 ... Loss 0.350 ... Accuracy 0.812
Saved checkpoint as: model_state_dumps/20190619_194634/SimpleCNN.pth
... Iteration 112410 ... Epoch 5.00 ... Loss 0.355 ... Accuracy 0.750
... Iteration 112420 ... Epoch 5.00 ... Loss 0.508 ... Accuracy 0.750
... Iteration 112430 ... Epoch 5.00 ... Loss 0.275 ... Accuracy 0.812
... Iteration 112440 ... Epoch 5.00 ... Loss 0.378 ... Accuracy 0.812
... Iteration 112450 ... Epoch 5.00 ... Loss 0.330 ... Accuracy 0.812
... Iteration 112460 ... Epoch 5.00 ... Loss 0.331 ... Accuracy 0.781
... Iteration 112470 ... Epoch 5.00 ... Loss 0.431 ... Accuracy 0.812
... Iteration 112480 ... Epoch 5.00 ... Loss 0.389 ... Accuracy 0.750
... Iteration 112490 